In [4]:
# from hindi_model import model_predict
import pandas as pd

In [13]:
import pickle
from gensim.models import Word2Vec
import numpy as np
import xgboost

# prefix = "Emotion Detection 2/"
prefix = ""

with open(f"{prefix}xgboost_model.pkl", "rb") as f:
    model = pickle.load(f)

tokenizer = Word2Vec.load(f"{prefix}wordEmbedding_model.gensim")

def sent_vectorized(sentence: list[str]):
    wv_res=np.zeros(tokenizer.wv.vector_size)
    ctr=1
    for word in sentence:
        if word in tokenizer.wv:
            ctr +=1
            wv_res+=tokenizer.wv[word]
    wv_res=wv_res/ctr
    return wv_res

def model_predict(txt: str):
    avg_vector = sent_vectorized(txt.split())
    avg_vector = np.expand_dims(avg_vector, axis=0)
    dmatrix = xgboost.DMatrix(avg_vector)
    return model.predict(dmatrix)[0]

In [ ]:
def getScores(data_file) -> dict[str, list[float]]:
    df = pd.read_csv(data_file, escapechar="\\", skipinitialspace=True)
    df.sort_values(by="product_id", inplace=True)
    df.set_index(keys=["product_id"], drop=False, inplace=True)
    products = df["product_id"].unique().tolist()
    result: dict[str, list[float]] = {}
    for product in products:
        prod_df = df.loc[df["product_id"] == product]
        scores = [0, 0, 0] # Index 0 = Positive, 1 = Negative, 2 = Neutral
        for _, row in prod_df.iterrows():
            emotion = int(model_predict(row["review"]))
            scores[emotion] += 1
        print(scores)
        # scores = np.average(scores, axis=0)
        result[product] = scores
    print(result)
    return result

In [14]:
with open("hindi_reviews_50.csv", "rb") as f:
    reviews = pd.read_csv(f)

scores = [0, 0, 0]
for _, row in reviews.iterrows():
    emotion = int(model_predict(row['review']))
    scores[emotion] += 1

scores

[15, 7, 29]

In [2]:
model_predict("यह सुपरमार्केट के कुछ ब्रांड्स से थोड़ा ज़्यादा पौष्टिक लगता है, लेकिन यह थोड़ा चिपचिपा है और इसमें उतना स्वाद भी नहीं है। यह मेरे बच्चों को पसंद नहीं आया, इसलिए मैं शायद इसे दोबारा नहीं खरीदूँगा।")

2.0